In [4]:
import numpy as np
import pandas as pd
from keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense,BatchNormalization, Flatten, MaxPool2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from keras.layers import Conv2D, Reshape
from keras.backend import epsilon
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
import cv2
import os

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
train_csv = pd.read_csv('/content/drive/MyDrive/deep learning/hack/train_(4).csv', dtype='str')
test_csv = pd.read_csv('/content/drive/MyDrive/deep learning/hack/test_(3).csv', dtype='str')

In [6]:
train_path='/content/drive/MyDrive/deep learning/hack/train-20211016T075959Z-001/train/'
test_path='/content/drive/MyDrive/deep learning/hack/test-20211014T130700Z-001/test'

In [24]:
data_with_aug = ImageDataGenerator(horizontal_flip=True,
                                   vertical_flip=False,
                                   rescale=1./255,
                                  validation_split=0.2)
train= data_with_aug.flow_from_directory(train_path,
                                          class_mode="binary",
                                          target_size=(150, 150),
                                          batch_size=32,
                                          subset="training")
test= data_with_aug.flow_from_directory(train_path,
                                          class_mode="binary",
                                          target_size=(150, 150),
                                          batch_size=32,
                                          subset="validation"
                                          )

Found 1368 images belonging to 2 classes.
Found 341 images belonging to 2 classes.


In [13]:

model = keras.Sequential()

# Convolutional layer and maxpool layer 1
model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 2
model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 3
model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 4
model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# This layer flattens the resulting image array to 1D array
model.add(keras.layers.Flatten())

# Hidden layer with 512 neurons and Rectified Linear Unit activation function 
model.add(keras.layers.Dense(512,activation='relu'))

# Output layer with single neuron which gives 0 for Cat or 1 for Dog 
#Here we use sigmoid activation function which makes our model output to lie between 0 and 1
model.add(keras.layers.Dense(1,activation='sigmoid'))


In [25]:

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [26]:

#steps_per_epoch = train_imagesize/batch_size

model.fit_generator(train,
         steps_per_epoch = 20,
         epochs = 5,
         validation_data = test
       
         )


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
20/20 [==============================] - 43s 2s/step - loss: 0.6606 - accuracy: 0.6156 - val_loss: 0.6758 - val_accuracy: 0.5953
Epoch 2/5
20/20 [==============================] - 42s 2s/step - loss: 0.6394 - accuracy: 0.6484 - val_loss: 0.6868 - val_accuracy: 0.5865
Epoch 3/5
20/20 [==============================] - 41s 2s/step - loss: 0.6422 - accuracy: 0.6155 - val_loss: 0.6428 - val_accuracy: 0.6422
Epoch 4/5
20/20 [==============================] - 42s 2s/step - loss: 0.6155 - accuracy: 0.6500 - val_loss: 0.7061 - val_accuracy: 0.6070
Epoch 5/5
20/20 [==============================] - 41s 2s/step - loss: 0.6265 - accuracy: 0.6187 - val_loss: 0.6670 - val_accuracy: 0.6158


In [27]:
test_img = pd.read_csv('/content/drive/MyDrive/deep learning/hack/test_(3).csv', dtype='str')
test_img['file_id']=test_img['file_id']+'.jpg'
test_img.shape

(332, 1)

In [31]:
main_test = data_with_aug.flow_from_dataframe(
    test_img,
    directory=test_path,
    seed= 42,
    x_col='file_id',
    y_col=None,
    target_size = (150,150),
    color_mode = 'rgb',
    class_mode=None,
    shuffle=False,
    batch_size= 32,    
)

Found 332 validated image filenames.


In [32]:
y_pred = model.predict(main_test)

In [33]:
y_predargmax = tf.argmax(y_pred , 1)
print(y_predargmax)

tf.Tensor(
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], shape=(332,), dtype=int64)
